In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import cv2, os
import tensorflow as tf
from tensorflow.keras import  layers, models
from sklearn.model_selection import train_test_split

In [2]:
def set_seed(seed=2137):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
set_seed()
DATA_PATH = 'data/landscape/'
IMG_DIM =(224,224)

In [3]:
def augment_data(img_l,img_ab):
    image=tf.concat([img_l,img_ab],axis=-1)
    image = tf.image.random_flip_left_right(image)
    image = tf.image.rot90(image, k=tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32))

    
    image_l = image[..., :1]
    image_ab = image[..., 1:]

    return image_l, image_ab
def create_dataset(X,y,batch_size=32, augmentation=False):
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    dataset = dataset.shuffle(1000)
    if augmentation:
        dataset = dataset.map(augment_data, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset
    

In [4]:
def display_colorized_images(model, X_test, y_test=None, num_images=10):
    indices = np.random.choice(len(X_test), num_images, replace=False)
    plt.figure(figsize=(15, num_images * 3)) 
    
    for i, idx in enumerate(indices):
        img_l = X_test[idx][np.newaxis, ...]  
        img_l_display = img_l[0, ..., 0] * 255  
        
        
        pred_ab = model.predict(img_l, verbose=0)[0] 
        pred_ab = (pred_ab + 1) * 128 
        
        
        img_lab = np.zeros((224, 224, 3), dtype=np.float32)
        img_lab[:, :, 0] = img_l_display
        img_lab[:, :, 1:] = pred_ab
        
        
        img_rgb = cv2.cvtColor(img_lab.astype(np.uint8), cv2.COLOR_LAB2RGB)
        
        
        plt.subplot(num_images, 3, i * 3 + 1)
        plt.title("Grayscale (L)")
        plt.imshow(img_l_display, cmap='gray')
        plt.axis('off')
        
        
        plt.subplot(num_images, 3, i * 3 + 2)
        plt.title("Predicted Color")
        plt.imshow(img_rgb)
        plt.axis('off')
        
        
        if y_test is not None:
            img_ab_true = (y_test[idx] + 1) * 128  
            img_lab_true = np.zeros((224, 224, 3), dtype=np.float32)
            img_lab_true[:, :, 0] = img_l_display
            img_lab_true[:, :, 1:] = img_ab_true
            img_rgb_true = cv2.cvtColor(img_lab_true.astype(np.uint8), cv2.COLOR_LAB2RGB)
            
            plt.subplot(num_images, 3, i * 3 + 3)
            plt.title("Ground Truth")
            plt.imshow(img_rgb_true)
            plt.axis('off')
    
    plt.tight_layout()
    plt.show()

In [ ]:
X = []
y = []
for imgdir in os.listdir(DATA_PATH):
    img = cv2.imread(DATA_PATH + imgdir)
    img_lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    img_lab_rs = cv2.resize(img_lab,IMG_DIM)
    img_l = img_lab_rs[:,:,0]
    img_ab = img_lab_rs[:,:,1:] 
    X.append(img_l[...,np.newaxis]) # [...,np.newaxis] sprawia że mamy tensor (128,128,1) a nie (128,128)
    y.append(img_ab)
    
X = np.array(X)
y = np.array(y)
X = X /255
y = y / 128 -1 


In [ ]:
X_train,X_test, y_train,y_test = train_test_split(X, y,test_size=0.2)
input_shape = X_train[0].shape
print(input_shape)

In [ ]:
model = tf.keras.Sequential([
    layers.InputLayer(shape=input_shape),

    
  layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_uniform'),
    layers.Conv2D(64, (3, 3), activation='relu', padding='same', strides=2, kernel_initializer='he_uniform'),
    layers.BatchNormalization(),
    
    layers.Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer='he_uniform'),
    layers.Conv2D(128, (3, 3), activation='relu', padding='same', strides=2, kernel_initializer='he_uniform'),
    layers.BatchNormalization(),
    
   
    layers.Conv2DTranspose(64, (4, 4), strides=2, activation='relu', padding='same', kernel_initializer='he_uniform'),
    layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_uniform'),
    layers.BatchNormalization(),
    
    layers.Conv2DTranspose(32, (4, 4), strides=2, activation='relu', padding='same', kernel_initializer='he_uniform'),
    layers.Conv2D(32, (3, 3), activation='relu', padding='same', kernel_initializer='he_uniform'),
    layers.BatchNormalization(),
    
    layers.Conv2D(2, (3, 3), activation='tanh', padding='same', kernel_initializer='he_uniform'),
])

In [ ]:

model.compile(
    optimizer='adam',
    loss='mean_squared_error',
)
model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ReduceLROnPlateau(patience=3, factor=0.5)
]

In [ ]:
train_ds = create_dataset(X_train, y_train, batch_size=32, augmentation=True)
val_ds = create_dataset(X_test, y_test, batch_size=32, augmentation=False)

history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=50,
                    callbacks=callbacks,
                   verbose=1)